In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', 100)

In [35]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as po

In [11]:
data = pd.read_csv('맥주_10_732개.csv', encoding='utf-8', index_col=0)
tmp = data.copy()

C:\Users\Ghyeon\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
tmp.head(3)

,아이디,맥주,날짜,평점,편의점,Aroma,Appearance,Flavor,Mouthfeel,Overall
2,"snoworsummer(8,581)",Kloud Original Gravity,"March 7, 2020",3.0,CU GS25 7/11 ministop,6,3,6,3,12
3,mickmac(726),Kloud Original Gravity,"February 20, 2020",2.9,CU GS25 7/11 ministop,5,4,4,4,12
4,"GuideDogSaint(2,549)",Kloud Original Gravity,"February 8, 2020",2.2,CU GS25 7/11 ministop,4,2,5,2,9


In [16]:
# 데이터 전처리
tmp = tmp[['맥주']]
tmp.head(3)

# 중복행 제거 : df.drop_duplicates()
tmp.drop_duplicates(keep='first', inplace=True)

# Column 값 처리
cols = ['Aroma','Appearance','Flavor','Mouthfeel','Overall']
for col in cols:
    tmp[col] = ''

for i in range(len(tmp)):
    beer = tmp['맥주'].iloc[i]
    
    length = len(data[data['맥주']==beer])
    for col in cols:
        col_sum = data[data['맥주']==beer][col].sum()
        tmp[col].iloc[i] = col_sum/length

In [18]:
tmp.to_csv('맥주_10_종류별평점.csv', encoding='utf-8')

## 실제 클러스터링

In [65]:
data = pd.read_csv('맥주이름.csv', encoding='utf-8', index_col=0)
data.to_excel('맥주이름.xlsx')

In [20]:
# 데이터 불러오기
data = pd.read_csv('맥주_10_종류별평점.csv', encoding='utf-8',index_col=0)
data.head(3)

,맥주,Aroma,Appearance,Flavor,Mouthfeel,Overall
2,Kloud Original Gravity,4.333333,2.821429,4.464286,2.500000,8.904762
0,Kloud Fitz Super Clear,3.571429,2.190476,3.238095,1.904762,6.761905
1,Asahi Super Dry,3.858397,2.584552,4.210064,2.407841,8.344061


In [21]:
scaler = MinMaxScaler()

# value에 해당하는 값만 추출
beer_names = data[['맥주']]
beer_values = data[['Aroma','Appearance','Flavor','Mouthfeel','Overall']]

# 값 정규화
scaler.fit(beer_values)
scaled = scaler.transform(beer_values)

# 클러스터링 예측 결과
km = KMeans(n_clusters=3).fit(scaled)
predict = pd.DataFrame(km.predict(scaled))
predict.columns = ['Cluster']

# 데이터프레임 변환
scaled = pd.DataFrame(data=scaled, columns=['Aroma', 'Appearance', 'Flavor', 'Mouthfeel','Overall'])
beer_names.reset_index(inplace=True, drop=True)
# scale 값과 클러스터링 예측값, 기존 맥주이름, 편의점 데이터 병합
result = pd.concat([scaled, beer_names], axis=1).reset_index(drop=True)
result = pd.concat([result, predict], axis=1).reset_index(drop=True)

# 값 추출
c_result = km.cluster_centers_
c_result = pd.DataFrame(data=c_result)
c_result.columns = ['Aroma', 'Appearance', 'Flavor','Mouthfeel','Overall']

In [22]:
result

,Aroma,Appearance,Flavor,Mouthfeel,Overall,맥주,Cluster
0,0.407336,0.462508,0.419377,0.437815,0.429697,Kloud Original Gravity,2
1,0.283196,0.241671,0.219705,0.224594,0.267660,Kloud Fitz Super Clear,1
2,0.329953,0.379600,0.377980,0.404803,0.387298,Asahi Super Dry,1
3,0.303618,0.305380,0.349956,0.368434,0.349176,Tsingtao,1
4,0.292305,0.368255,0.362223,0.387053,0.371668,Heineken,1
...,...,...,...,...,...,...,...
597,0.296352,0.311962,0.301293,0.305433,0.325117,Desant Svitle (DMB),1
598,0.399577,0.419452,0.433378,0.446344,0.444100,Slavutych Pivo (Slavutich Premium),2
599,0.400623,0.458154,0.438305,0.439828,0.436332,Kasztelan Niepasteryzowane,2
600,0.368300,0.390111,0.387875,0.389308,0.394364,Brok Export,1


In [24]:
c_result.values

array([[0.74783511, 0.70897644, 0.75901063, 0.72578077, 0.76374302],
       [0.30489174, 0.34994392, 0.31696907, 0.34157723, 0.3172393 ],
       [0.48227848, 0.4968135 , 0.49964651, 0.49838448, 0.50025337]])

In [25]:
c_result = pd.DataFrame(data=c_result, columns=['Aroma', 'Appearance', 'Flavor','Mouthfeel','Overall'])

In [27]:
c_result.sort_values(by='Overall', inplace=True)

In [45]:
c_result.values

array([[0.30489174, 0.34994392, 0.31696907, 0.34157723, 0.3172393 ],
       [0.48227848, 0.4968135 , 0.49964651, 0.49838448, 0.50025337],
       [0.74783511, 0.70897644, 0.75901063, 0.72578077, 0.76374302]])

In [57]:
target = result[result['맥주'] == 'Cass Fresh']

In [58]:
target = target[['Aroma', 'Appearance', 'Flavor','Mouthfeel', 'Overall']]
target = target.values[0]
target

array([0.25087721, 0.2746201 , 0.25107951, 0.28213075, 0.24872818])

In [60]:
target

array([0.25087721, 0.2746201 , 0.25107951, 0.28213075, 0.24872818])

In [43]:
c_result.to_csv('Cluster_군집결과.csv', encoding='utf-8')

In [44]:
result.to_csv('Clusering_전체결과.csv', encoding='utf-8')

In [40]:
def show_cluster(c_result, result, name):
    categories = ['Aroma', 'Appearance', 'Flavor','Mouthfeel', 'Overall']
    color = ['skyblue', 'blue', 'salmon', 'green']
    
    target = result[result['맥주'] == name]
    cluster = target['Cluster'].iloc[0]
    target = target[categories]
    
#     matching = {0:'Soso', 1:'Sad', 2:'Good'}
#     print('{0} 맥주는 {1} 유형입니다. \n\n'.format(name,matching[cluster]))
    
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
#         r = c_result.values[0],
        r = c_result.iloc[0],
        theta = categories,
        fill='toself',
        name='Sad',
        line_color=color[0]
    ))

    fig.add_trace(go.Scatterpolar(
#         r = c_result.values[1],
        r = c_result.iloc[1],
        theta = categories,
        fill='toself',
        name='Soso',
        line_color=color[1]
    ))

    fig.add_trace(go.Scatterpolar(
#         r = c_result.values[2],
        r = c_result.iloc[2],
        theta = categories,
        fill='toself',
        name='Good',
        line_color=color[2]
    ))
    
    fig.add_trace(go.Scatterpolar(
        r = target.values[0],
        theta = categories,
        fill='toself',
        name=name,
        line_color=color[3]
    ))

    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
#           range=[0, 1]
        )),
    )
    
    fig.show()

In [41]:
show_cluster(c_result, result, 'Kloud Original Gravity')